In [1]:
from dotenv import load_dotenv
import os
import openai
from datasets import load_dataset
import llama_index
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import PineconeVectorStore
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
import chromadb
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.prompts import PromptTemplate
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.node_parser import SimpleNodeParser
from pathlib import Path
from llama_index import download_loader
from llama_index.embeddings import OpenAIEmbedding
from llama_index.text_splitter import SentenceSplitter
from llama_index.extractors import TitleExtractor
from llama_index.ingestion import IngestionPipeline, IngestionCache
from llama_index import download_loader
from pathlib import Path
from llama_index import download_loader
from llama_index import VectorStoreIndex
from llama_index import SummaryIndex
import openai
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    get_response_synthesizer,
)
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)

from llama_index.ingestion import IngestionPipeline
from llama_index.text_splitter import TokenTextSplitter

from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [2]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [13]:
storage_context = StorageContext.from_defaults(
    docstore= SimpleDocumentStore.from_persist_dir(persist_dir="C:/Users/jrdef/Documents/ds384"),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="C:/Users/jrdef/Documents/ds384")
)

ds384 = load_index_from_storage(storage_context)

In [35]:
PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path('C:/Users/jrdef/Downloads/civil_war.pdf'))

In [36]:
pages = documents[1:3]

## BEGIN TO MAKE FLASHCARD

In [19]:
def get_llm(openai_api_key, max_tokens=8192):
    os.environ["OPENAI_API_KEY"] = openai_api_key
    return OpenAI(
        temperature=0.0, model='gpt-3.5-turbo-16k', max_tokens=max_tokens
    )

In [20]:
llm = get_llm(OPENAI_API_KEY)


In [34]:
def get_ds_index(documents, llm, api_key):

    service_context = ServiceContext.from_defaults(llm=llm, chunk_size=384, chunk_overlap=128)

    response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
    )
    
    temp_index = DocumentSummaryIndex.from_documents(
        documents, 
        service_context=service_context,
        response_synthesizer = response_synthesizer
    )

    return temp_index

In [39]:
ds384 = get_ds_index(pages, llm, OPENAI_API_KEY)

current doc id: 4782c65f-d7f1-43b3-a89a-ef8e9ccebf14
current doc id: 7cdd6ed8-a751-4238-88eb-c5278b9457cd


In [85]:
def extract_terms_1(index, prompt, summary, k=1):
    query_engine = index.as_query_engine(
        similarity_top_k=k
    )

    f_prompt = prompt.format(summary)

    terms_definitions = str(query_engine.query(f_prompt))
    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }
    return terms_to_definition

In [83]:
def summarize_docs(index, prompt, k=1):
    query_engine = index.as_query_engine(similarity_top_k=k)

    summary = str(query_engine.query(prompt))

    return summary


In [37]:
def extract_terms(index, summary, prompts, k=1):
    query_engine = index.as_query_engine(similarity_top_k=k)

    all_terms_to_definition = {}
    for prompt in prompts:
        terms_definitions = str(query_engine.query(prompt))
        terms_definitions = [
            x
            for x in terms_definitions.split("\n")
            if x and "Term:" in x and "Definition:" in x
        ]
        for term_definition in terms_definitions:
            term = term_definition.split("Definition:")[0].split("Term:")[-1].strip()
            definition = term_definition.split("Definition:")[-1].strip()

            if term in all_terms_to_definition:
                # Combine definitions if they are different
                if all_terms_to_definition[term] != definition:
                    all_terms_to_definition[term] += " Additionally, " + definition
            else:
                all_terms_to_definition[term] = definition

    return all_terms_to_definition

In [56]:
def check_terms(index, context_str, prompt, k = 1):
    query_engine = index.as_query_engine(similarity_top_k = k)

    context_str = str(context_str)

    final_prompt = prompt.format(context_str)

    terms_definitions = str(query_engine.query(final_prompt))
    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }
    return terms_to_definition

    

In [98]:
summaryPrompt = (
    """Task: Carefully read the document and produce a concise summary. 
    It is crucial to highlight all main themes, crucial events, key figures, and significant concepts, as missing any could have dire consequences.
    ensure that you mention all people and events of importance that had any significant context in the field.
    Ensure the summary distills the document's content into its most essential elements, capturing every potential term and idea without omission. 
    The accuracy and comprehensiveness of this summary are vital, as if a life depends on it, setting the stage for accurate term and definition extraction in the next phase."""
)

summaryPrompt2 = (
    """Task: Create a detailed summary of the document, emphasizing all major events, crucial figures, and central themes. 
    It is imperative that no significant event names or key individuals are omitted. 
    After summarizing, list the most important terms derived from this reading, such as specific events, people, concepts, and terms crucial to understanding the subject matter. 
    This list will serve as a vital reference for extracting precise terms and definitions in subsequent stages."""
)

In [99]:
s1 = summarize_docs(ds384, summaryPrompt2)

In [100]:
s1

'The document provides information about the Civil War, including major events, key figures, and central themes. The Confederacy fought a defensive war after suffering significant defeats in 1863, until their armies surrendered to Union troops. Some important events mentioned in the document include the battles of Gettysburg, Petersburg, and Atlanta, as well as the surrender at Appomattox Court House. The document also highlights the regions where most of the fighting was concentrated after 1863, and the devastating impact of the war on both sides. The Union costs of the war were $6.1 billion with an inflation rate of 80 percent, while the Confederate costs were $2 billion with an inflation rate of 9,000 percent. Many southern farms, factories, and railroads were destroyed, and the southern industry was crippled. The Confederate states lost two-thirds of their wealth. The death toll from the Civil War was staggering, with over half a million soldiers killed and more than 500,000 wounde

In [78]:
prompt_list = [bossPrompt, megaprompt, prompt2]

In [101]:
t1 = extract_terms_1(ds384, term_prompt, s1, 3)

In [102]:
t1

{'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Confederacy': 'The group of Southern states that seceded from the United States and formed their own government during the Civil War.',
 'Union troops': 'The military forces of the Northern states during the Civil War.',
 'Gettysburg': 'A major battle fought in Gettysburg, Pennsylvania, in July 1863, considered a turning point in the Civil War.',
 'Petersburg': 'A city in Virginia where a significant battle took place during the Civil War, known as the Siege of Petersburg.',
 'Atlanta': 'A city in Georgia that was a major target for Union forces during the Civil War, and was eventually captured in September 1864.',
 'Appomattox Court House': 'The location in Virginia where Confederate General Robert E. Lee surrendered to Union General Ulysses S. Grant, effectively ending the Civil War.',
 'Defensive war': 'A war strategy in which a side prima

In [104]:
c2

{'Civil War': 'A war fought between the Northern states (Union) and the Southern states (Confederacy) in the United States from 1861 to 1865.',
 'Confederacy': 'The group of Southern states that seceded from the United States and formed their own government during the Civil War.',
 'Union troops': 'The military forces of the Northern states during the Civil War.',
 'Gettysburg': 'A major battle fought in Gettysburg, Pennsylvania, in July 1863, considered a turning point in the Civil War.',
 'Petersburg': 'A city in Virginia where a significant battle took place during the Civil War, known as the Siege of Petersburg.',
 'Atlanta': 'A city in Georgia that was a major target for Union forces during the Civil War, and was eventually captured in September 1864.',
 'Appomattox Court House': 'The location in Virginia where Confederate General Robert E. Lee surrendered to Union General Ulysses S. Grant, effectively ending the Civil War.',
 'Defensive war': 'A war strategy in which a side prima

In [79]:
d14= extract_terms(ds384, prompt_list, 2)  # Here, 3 is the similarity_top_k value

In [103]:
c2 = check_terms(ds384, t1, finalCheckPrompt)

In [80]:
d14

{'Union': 'A side in the Civil War that fought to preserve the United States and abolish slavery.',
 'Confederate': 'A side in the Civil War that fought for the independence of the Southern states and the preservation of slavery.',
 'Union forces': 'The military forces of the Union side in the Civil War.',
 'Confederate forces': 'The military forces of the Confederate side in the Civil War.',
 'Union victory': 'A successful outcome for the Union side in a battle or the overall war.',
 'Confederate victory': 'A successful outcome for the Confederate side in a battle or the overall war.',
 'Gettysburg': 'A town in Pennsylvania where a major battle of the Civil War took place in 1863.',
 'Petersburg': 'A city in Virginia where a major battle of the Civil War took place in 1864-1865.',
 'Atlanta': 'A city in Georgia that was a key target for the Union forces during the Civil War.',
 'Appomattox Court House': 'A village in Virginia where General Robert E. Lee surrendered to General Ulysses 

In [82]:
c1

{'Union': 'A side in the Civil War that fought to preserve the United States and abolish slavery.',
 'Confederate': 'A side in the Civil War that fought for the independence of the Southern states and the preservation of slavery.',
 'Union forces': 'The military forces of the Union side in the Civil War.',
 'Confederate forces': 'The military forces of the Confederate side in the Civil War.',
 'Union victory': 'A successful outcome for the Union side in a battle or the overall war.',
 'Confederate victory': 'A successful outcome for the Confederate side in a battle or the overall war.',
 'Gettysburg': 'A town in Pennsylvania where a major battle of the Civil War took place in 1863.',
 'Petersburg': 'A city in Virginia where a major battle of the Civil War took place in 1864-1865.',
 'Atlanta': 'A city in Georgia that was a key target for the Union forces during the Civil War.',
 'Appomattox Court House': 'A village in Virginia where General Robert E. Lee surrendered to General Ulysses 

In [91]:
term_prompt = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "For terms that are similar dont use the same term twice meaning if you use Union, there is no need for another term Union forces"
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"

    "the following is a summary of the context so you can have a better understanding of what is important within the text"
    "{}"
)


In [ ]:
prompt8 = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line."
    "there will be a name 'Mathew Brady' this needs to be in the dictionary or i die"
    "Note that formal names, like 'Mathew Brady', 'Abraham Lincoln' or 'Michael Duke', should be terms and should be given a definition. "
    "If a term is missing its definition, use your best judgment. "
    "With each term-definition pair on a separate line, be vigilant to not miss any names, regardless of their perceived prominence or importance. "
    "This task is crucial, as if my life depends on it, so please ensure the utmost accuracy and relevance. "
    "Write each line as follows:\nTerm: <term> Definition: <definition>"
)
prompt2 = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "For terms that are similar dont use the same term twice meaning if you use Union, there is no need for another term Union forces"
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
)


In [73]:
finalCheckPrompt = (
    """
    With the following string which contains a text representation of a dictionary of terms. this is the string {}.
    Ensure that each term is directly relevant to the document's subject matter. 
    Eliminate any redundant terms or definitions, retaining only unique and necessary information. 
    Verify that no significant terms or concepts have been overlooked. 
    if there is a group talked about more than once, ensure that the definition is already accurately represented and do not repeat.
    The goal is to refine the output into a perfectly concise and comprehensive list of terms and definitions, free from repetition and irrelevant content. 
    This means that for each term, ensure there is a definition in the text which matches and ensures that this is relevant to the context. Use relevant information you know in order to do this. 
    Your meticulous attention to detail is vital for the integrity and utility of the final output.
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"

    Begin the final review and refinement process:"""
)

In [23]:
prompt_general = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "Focus on capturing a wide range of terms, ensuring no significant concept, name, or entity is overlooked. "
    "For each term, provide a clear and succinct definition that encapsulates its essence within the context. "
    "In cases where the definition of a term is not explicitly stated, use contextual clues to derive an appropriate meaning. "
    "This task is crucial for understanding the broader scope of the subject matter. "
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
)

prompt_detailed = (
    "Construct a detailed list of terms, particularly focusing on names, places, events, and other specific categories, along with their definitions. "
    "Ensure that each term, especially proper nouns and technical terms, is accompanied by a precise and comprehensive definition. "
    "Leverage the context to discern nuanced meanings and subtle distinctions between similar terms. "
    "This task is essential for a deep and accurate understanding of the specific elements within the subject matter. "
    "Maintain clarity in your list by following this structure for each entry:\nTerm: <term> Definition: <definition>"
)

In [76]:
megaprompt = (
    "Carefully review the document and compile an exhaustive list of terms and their definitions, "
    "presenting each term-definition pair on a separate line. "
    "Prioritize identifying a diverse range of terms, especially focusing on proper nouns such as 'Mathew Brady', 'Abraham Lincoln', key technical terms, concepts, and significant entities. "
    "For each term, provide a definition that is clear, concise, and contextually derived when not explicitly stated. "
    "Special emphasis should be on nuanced meanings and subtle distinctions. "
    "Avoid redundancy in terms - for instance, if 'Union' is already listed, do not include 'Union victory' unless it adds distinct, new information. "
    "This task demands exceptional accuracy and thoroughness due to its critical nature. "
    "In ambiguous cases, employ informed judgment for the most fitting definitions. "
    "Aim for a deep, accurate understanding and representation of the material, steering clear of irrelevant details like page labels or file names. "
    "Format your response as:\nTerm: <term> Definition: <definition>"
)

prompt = (
    "completely understand what the material is about in terms of learning objective"
    "Find important information that would pertain to learning of the topic, those would be defined as terms"
    "Dont add any terms that take away from the learning objective and "
    "When sifting through the definitions and terms if you find very high similarities in definitions just combine the terms so that it can explain everything in one term"
)

In [77]:
bossPrompt = (
    """Task: Analyze the results from the following prompts. 
    For each prompt, identify the key terms extracted along with their definitions. 
    Compare the terms from both prompts, focusing on their relevance to the overarching topic. 
    Synthesize the information by selecting the most pertinent terms from each prompt. 
    Combine the knowledge from these terms to create a comprehensive understanding of the subject. 
    Ensure the final output is concise, accurate, and presents an integrated perspective based on the information from both prompts.
    
    Begin analysis and synthesis of the prompt results:"""
)

In [8]:
GBTPrompt = (
    """Task: Extract key terms and their definitions from the document. 
    For each term, especially proper nouns and technical terms, provide a clear definition. 
    Avoid redundancy; do not list variations of the same term. 
    Include notable names like 'Mathew Brady' and 'Abraham Lincoln' without fail. 
    Use context to infer meanings for ambiguous terms. 
    Desired Output Format:
    1. Term: <term>, Definition: <definition>
    Example:
    Term: Union, Definition: The northern states during the Civil War.
    Term: Abraham Lincoln, Definition: 16th President of the United States, led the nation during the Civil War.
    
    Begin extraction from the document below:"""
)

In [44]:
# Create a list of prompts
# Call the function with your index and prompts
d14= extract_terms_1(ds384, prompt2, 2)  # Here, 3 is the similarity_top_k value

In [45]:
d14

{'Union': 'A side in the Civil War that fought to preserve the United States and abolish slavery.',
 'Confederate': 'A side in the Civil War that fought for the independence of the Southern states and the preservation of slavery.',
 'Union forces': 'The military forces of the Union side in the Civil War.',
 'Confederate forces': 'The military forces of the Confederate side in the Civil War.',
 'Union victory': 'A successful outcome for the Union side in the Civil War.',
 'Confederate victory': 'A successful outcome for the Confederate side in the Civil War.',
 'Gettysburg': 'A major battle in the Civil War that took place in Gettysburg, Pennsylvania.',
 'Petersburg': 'A city in Virginia that was the site of a major battle in the Civil War.',
 'Atlanta': 'A city in Georgia that was the site of a major battle in the Civil War.',
 'Appomattox Court House': 'The location where General Robert E. Lee surrendered to General Ulysses S. Grant, effectively ending the Civil War.',
 'defensive war